# **Importing and Extracting the dataset**

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import classification_report
import seaborn as sns

In [2]:
train_path = r'aptos_augmented_images_resized/train'
test_path = r'aptos_augmented_images_resized/test'

In [4]:
training_data = tf.keras.utils.image_dataset_from_directory(train_path,validation_split=0.1,interpolation='area',image_size=(128,128),batch_size=16,subset='training',seed=42,color_mode='rgb')
validationData = tf.keras.utils.image_dataset_from_directory(train_path,validation_split=0.1,interpolation='area',image_size=(128,128),batch_size=16,subset='validation',seed=42,color_mode='rgb')
testing_data = tf.keras.utils.image_dataset_from_directory(test_path,interpolation='area',image_size=(128,128),shuffle=False,batch_size=16)

Found 8000 files belonging to 5 classes.
Using 7200 files for training.
Found 8000 files belonging to 5 classes.
Using 800 files for validation.
Found 2000 files belonging to 5 classes.


In [5]:
#Creating list for y_test

test_labels = []
for i in range(0,5):
    for j in range(0,400):
        test_labels.append(i)
y_test = np.array(test_labels,dtype='int8')

# **Building and Training Individual Models**

In [6]:
def model_building(base_model):

    for layer in base_model.layers:
        layer.trainable=False

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Rescaling(scale=1./255))
    model.add(tf.keras.layers.RandomFlip("horizontal_and_vertical"))
    model.add(tf.keras.layers.RandomRotation(0.2))
    model.add(base_model)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(500, activation='relu'))
    model.add(tf.keras.layers.Dense(100, activation='relu'))
    model.add(tf.keras.layers.Dense(5, activation='softmax'))

    return model

In [7]:
def model_training(model):
    
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'],run_eagerly=True)
    model_history = model.fit(training_data,shuffle=True,epochs=5,validation_data=validationData)

    return model_history

In [8]:
def save_model(model,name):
    model.save('models/' + name + '.h5')

# **Predictions of Individual Models**

In [ ]:
def model_prediction(model):
    return model.predict(testing_data)

# **Model Graphs**

In [ ]:
def epoch_vs_accuracy(model_history):
    
    plt.plot(model_history.history['accuracy'])
    plt.plot(model_history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
def classification_report(y_pred):

    return classification_report(y_true=y_test,y_pred=y_pred,labels=[0,1,2,3,4])

# **Metrics**

In [ ]:
def calculate_metrics(actual, predicted, class_labels):

    tn = np.zeros(len(class_labels))
    fp = np.zeros(len(class_labels))
    tp = np.zeros(len(class_labels))
    fn = np.zeros(len(class_labels))
    specificity = np.zeros(len(class_labels))

    for i, c in enumerate(class_labels):
        actual_c = actual == c
        predicted_c = predicted == c
        tn[i] = np.sum(np.logical_and(actual_c == False, predicted_c == False))
        fp[i] = np.sum(np.logical_and(actual_c == False, predicted_c == True))
        tp[i] = np.sum(np.logical_and(actual_c == True, predicted_c == True))
        fn[i] = np.sum(np.logical_and(actual_c == True, predicted_c == False))
        specificity[i] = tn[i] / (tn[i] + fp[i])

    return tn, fp, tp, fn, specificity

# **Inseption Resnet V2**

In [ ]:
ivr2 = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(input_shape=[256,256]+[3],weights= 'imagenet' ,include_top=False)
ivr2 = model_building(ivr2)
ivr2_history = model_training(ivr2)
ivr2_predictions = model_prediction(ivr2)
save_model(ivr2,'IRv2')

In [ ]:
epoch_vs_accuracy(ivr2_history)

In [ ]:
classification_report(ivr2_predictions)

In [ ]:
cm_ivr2 = tf.math.confusion_matrix(y_test,ivr2_predictions)
cm_ivr2

In [ ]:
tn, fp, tp, fn, specificity = calculate_metrics(y_test, ivr2_predictions, [0,1,2,3,4])
print('Inception Resnet V2 Specificity\n\n',specificity)

In [ ]:
sns.heatmap(cm_ivr2,fmt='d',annot=True)

# **Xception**

In [ ]:
xception = tf.keras.applications.xception.Xception(input_shape=[256,256]+[3],weights= 'imagenet' ,include_top=False)
xception = model_building(xception)
xception_history = model_training(xception)
xception_predictions = model_prediction(xception)
save_model(xception,'Xception')

In [ ]:
epoch_vs_accuracy(xception_history)

In [ ]:
classification_report(xception_predictions)

In [ ]:
cm_xception = tf.math.confusion_matrix(y_test,xception_predictions)
cm_xception

In [ ]:
tn, fp, tp, fn, specificity = calculate_metrics(y_test, xception_predictions, [0,1,2,3,4])
print('Xception Specificity\n\n',specificity)

In [ ]:
sns.heatmap(cm_xception,fmt='d',annot=True)

# **VGG19**

In [ ]:
vgg = tf.keras.applications.vgg19.VGG19(input_shape=[256,256]+[3],weights= 'imagenet' ,include_top=False)
vgg = model_building(vgg)
vgg_history = model_training(vgg)
vgg_predictions = model_prediction(vgg)
save_model(vgg,'VGG19')

In [ ]:
epoch_vs_accuracy(vgg_history)

In [ ]:
classification_report(vgg_predictions)

In [ ]:
cm_vgg = tf.math.confusion_matrix(y_test,vgg_predictions)
cm_vgg

In [ ]:
tn, fp, tp, fn, specificity = calculate_metrics(y_test, vgg_predictions, [0,1,2,3,4])
print('VGG19 Specificity\n\n',specificity)

In [ ]:
sns.heatmap(cm_vgg,fmt='d',annot=True)

# **Resnet**

In [ ]:
resnet = tf.keras.applications.vgg19.VGG19(input_shape=[256,256]+[3],weights= 'imagenet' ,include_top=False)
resnet = model_building(resnet)
resnet_history = model_training(resnet)
resnet_predictions = model_prediction(resnet)
save_model(resnet,'Resnet')

In [ ]:
epoch_vs_accuracy(resnet_history)

In [ ]:
classification_report(resnet_predictions)

In [ ]:
cm_resnet = tf.math.confusion_matrix(y_test,resnet_predictions)
cm_resnet

In [ ]:
tn, fp, tp, fn, specificity = calculate_metrics(y_test, resnet_predictions, [0,1,2,3,4])
print('Resnet Specificity\n\n',specificity)

In [ ]:
sns.heatmap(cm_resnet,fmt='d',annot=True)

# **Densenet**

In [ ]:
densenet = tf.keras.applications.vgg19.VGG19(input_shape=[256,256]+[3],weights= 'imagenet' ,include_top=False)
densenet = model_building(densenet)
densenet_history = model_training(densenet)
densenet_predictions = model_prediction(densenet)
save_model(densenet,'Densenet')

In [ ]:
epoch_vs_accuracy(densenet_history)

In [ ]:
classification_report(densenet_predictions)

In [ ]:
cm_densenet = tf.math.confusion_matrix(y_test,densenet_predictions)
cm_densenet

In [ ]:
tn, fp, tp, fn, specificity = calculate_metrics(y_test, densenet_predictions, [0,1,2,3,4])
print('Densenet Specificity\n\n',specificity)

In [ ]:
sns.heatmap(cm_densenet,fmt='d',annot=True)